In [1]:
import sys, platform
print("Python full:", sys.version)          # e.g. '3.12.5 (main, ...)'
print("Python tuple:", sys.version_info)     # e.g. sys.version_info(major=3, minor=12, micro=5, ...)
print("Python short:", platform.python_version())  # e.g. '3.12.5'
print("Executable:", sys.executable)         # path to the kernel's python


Python full: 3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 08:03:38) [Clang 14.0.6 ]
Python tuple: sys.version_info(major=3, minor=12, micro=11, releaselevel='final', serial=0)
Python short: 3.12.11
Executable: /opt/homebrew/anaconda3/envs/qiskit2x/bin/python


In [2]:
import qiskit, qiskit_ibm_runtime
print("qiskit:", qiskit.__version__)
print("runtime:", qiskit_ibm_runtime.__version__)


qiskit: 2.2.0
runtime: 0.41.1


In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token="iwtFvllG-Xzlu9b7235CWZ_-fG5X0dF4XS2lMudXuzyk",
    set_as_default=True,
    overwrite=True,
)
print("Saved IBM Quantum Platform account.")


Saved IBM Quantum Platform account.


In [4]:
# from qiskit_ibm_runtime import QiskitRuntimeService

# 1) Remove any saved accounts (clears the mismatch)
# for name in list(QiskitRuntimeService.saved_accounts().keys()):
#     QiskitRuntimeService.delete_account(name=name)

# # 2) Save your personal account on the Quantum Platform channel (no CRN)
# QiskitRuntimeService.save_account(
#     channel="ibm_quantum_platform",
#     token="PASTE_YOUR_API_TOKEN_HERE",
#     set_as_default=True,
#     overwrite=True,
# )
# print("Saved account on ibm_quantum_platform.")


In [5]:
# Prereq (run once, earlier): 
# from qiskit_ibm_runtime import QiskitRuntimeService
# QiskitRuntimeService.save_account(channel="ibm_quantum_platform", token="YOUR_API_TOKEN", set_as_default=True, overwrite=True)

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2

svc = QiskitRuntimeService()  # loads your saved account

# Pick a real device (or set one explicitly, e.g., backend_name = "ibm_brisbane")
real = [b for b in svc.backends() if not getattr(b.configuration(), "simulator", False) and b.status().operational]
if not real:
    raise RuntimeError("No operational real backends available to your account.")
backend_name = sorted(real, key=lambda b: (b.status().pending_jobs, b.configuration().num_qubits))[0].name
print("Using backend:", backend_name)

# Tiny Bell circuit with measurement
qc = QuantumCircuit(2, 2)
qc.h(0); qc.cx(0, 1); qc.measure([0, 1], [0, 1])

# Run with Runtime V2
with Session(service=svc, backend=backend_name) as session:
    job = SamplerV2(session=session).run([qc], shots=2048, seed=1234)
    print("Job ID:", job.job_id())
    res = job.result()

counts = res[0].data.meas.get_counts()
shots = sum(counts.values())
print("Total shots:", shots)
print({k: counts[k] for k in sorted(counts, key=counts.get, reverse=True)[:4]})


qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-09-20 13:32:51,606: Default instance not set. Searching all available instances.


IBMInputValueError: 'No matching instances found for the following filters: .'

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService
print("Saved accounts:", list(QiskitRuntimeService.saved_accounts().keys()))
for name, meta in QiskitRuntimeService.saved_accounts().items():
    print(name, "=>", {k: meta.get(k) for k in ("channel","instance","is_default")})


Saved accounts: ['default-ibm-quantum-platform']
default-ibm-quantum-platform => {'channel': 'ibm_quantum_platform', 'instance': None, 'is_default': None}


In [7]:
for name in list(QiskitRuntimeService.saved_accounts().keys()):
    QiskitRuntimeService.delete_account(name=name)